In [ ]:
# default_exp analysis.utils

# Utils for analysis

> Various methods to modify and work with python dictionaries

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

%load_ext autoreload
%autoreload 2

In [ ]:
# export 
import pandas as pd
import numpy as np
import torch

In [ ]:
# export 
def concat_preds(preds_list:list) -> tuple:
    '''Concatenate a list of fastai-style-like predictions into a single fastai-style-like prediction.
    
    Parameters
    ----------
    preds_list : list
        List of tuples where each tuple contains two tensors. 
        First tensor contains probabilities and second one ground truth. 
    
    Returns
    -------
    Fastai-style-like prediction
    '''
    probs = torch.cat(tuple([v[0] for v in preds_list]))
    gt = torch.cat(tuple([v[1] for v in preds_list]))
    
    return (probs, gt)

def ensemble(preds_list:list) -> tuple:
    '''Ensemble a list of fastai-style-like predictions into a single fastai-style-like prediction.
    
    Simple ensemble using averaging. 
    
    Parameters
    ----------
    preds_list : list
        List of tuples where each tuple contains two tensors. 
        First tensor contains probabilities and second one ground truth. 
    
    Returns
    -------
    Fastai-style-like prediction
    '''
    n = len(preds_list)
    
    probs, gt = preds_list[0][0].clone(), preds_list[0][1].clone()
    for preds in preds_list[1:]:
        assert torch.equal(gt, preds[1]), "Ground truth tensors do not match. Unequal size or elements."
        probs += preds[0]
    
    return (probs/n, gt)

def verify_softmax(preds:tuple):
    '''Verify softmax for a single fastai-style-like prediction.
    
    Parameters
    ----------
    preds : tuple
        Tuple with two tensors (fastai-style-like predictio). 
        First tensor contains probabilities and second one ground truth.  
    '''    
    assert torch.allclose(preds[0].sum(axis=1), torch.Tensor([1.])), "Softmax probabilities do not sum up to 1."